# Phase 2 — Statistical Analysis

**Activity 2.1 & 2.2 (Descriptive + Inferential)**

Notebook preparado para entregar la Fase 2. Incluye: EDA, medidas de tendencia, detección de outliers, análisis de distribución, data quality report, pruebas de hipótesis, correlaciones, clustering, modelos predictivos y análisis de series temporales.

**Archivos esperados**: `content.json`, `users.csv`, `viewing_sessions.csv` en `/mnt/data`.

---

In [ ]:
# Imports y configuración
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from scipy.cluster.hierarchy import linkage, dendrogram
%matplotlib inline

plt.rcParams['figure.figsize'] = (10,5)
sns.set_style('whitegrid')

DATA_PATH = '/mnt/data'
content_path = os.path.join(DATA_PATH, 'content.json')
users_path = os.path.join(DATA_PATH, 'users.csv')
sessions_path = os.path.join(DATA_PATH, 'viewing_sessions.csv')


In [ ]:
# Carga de datos
# content.json: puede contener 'movies' y 'series' o una lista de objetos
with open(content_path, 'r', encoding='utf-8') as f:
    content = json.load(f)

movies = pd.DataFrame(content.get('movies', []))
series = pd.DataFrame(content.get('series', []))
if not movies.empty:
    movies['content_type'] = 'movie'
if not series.empty:
    series['content_type'] = 'series'
content_df = pd.concat([movies, series], ignore_index=True, sort=False)

users_df = pd.read_csv(users_path)
sessions_df = pd.read_csv(sessions_path)

print('content_df shape:', content_df.shape)
print('users_df shape:', users_df.shape)
print('sessions_df shape:', sessions_df.shape)

# preview
display(content_df.head())
display(users_df.head())
display(sessions_df.head())

In [ ]:
# Limpieza y features derivadas
# Parseo de timestamps comunes
for col in ['start_time','end_time','started_at','ended_at','timestamp']:
    if col in sessions_df.columns:
        try:
            sessions_df[col] = pd.to_datetime(sessions_df[col])
        except Exception:
            pass

# session_duration_min si hay start & end
if ('start_time' in sessions_df.columns) and ('end_time' in sessions_df.columns):
    sessions_df['session_duration_min'] = (sessions_df['end_time'] - sessions_df['start_time']).dt.total_seconds() / 60.0

# normalizar completion_rate
if 'completion_rate' not in sessions_df.columns:
    if 'completion_percentage' in sessions_df.columns:
        sessions_df['completion_rate'] = sessions_df['completion_percentage'] / 100.0
    elif 'percent_watched' in sessions_df.columns:
        sessions_df['completion_rate'] = sessions_df['percent_watched'] / 100.0
    elif ('watched_seconds' in sessions_df.columns) and ('content_id' in sessions_df.columns):
        tmp = content_df.copy()
        if 'duration_minutes' in tmp.columns:
            tmp['content_duration_min'] = tmp['duration_minutes']
        elif 'avg_episode_duration' in tmp.columns:
            tmp['content_duration_min'] = tmp['avg_episode_duration']
        else:
            tmp['content_duration_min'] = np.nan
        tmp = tmp[['content_id','content_duration_min']]
        sessions_df = sessions_df.merge(tmp, on='content_id', how='left')
        sessions_df['completion_rate'] = sessions_df['watched_seconds'] / (sessions_df['content_duration_min']*60)

# clip completion_rate to [0,1]
if 'completion_rate' in sessions_df.columns:
    sessions_df['completion_rate'] = sessions_df['completion_rate'].clip(0,1)

print('Columns in sessions_df:', sessions_df.columns.tolist())

In [ ]:
# Estadísticas descriptivas
numeric_cols = sessions_df.select_dtypes(include=[np.number]).columns.tolist()
desc = sessions_df[numeric_cols].describe().T
display(desc)

# Central tendency & dispersion for key metrics
for m in ['session_duration_min','completion_rate']:
    if m in sessions_df.columns:
        print('\nMetric:', m)
        print('mean:', sessions_df[m].mean())
        print('median:', sessions_df[m].median())
        print('std:', sessions_df[m].std())
        print('min:', sessions_df[m].min(), 'max:', sessions_df[m].max())

In [ ]:
# Detección de outliers
if 'session_duration_min' in sessions_df.columns:
    col = 'session_duration_min'
    mean = sessions_df[col].mean()
    std = sessions_df[col].std()
    sessions_df['z_'+col] = (sessions_df[col] - mean) / std
    outliers = sessions_df[abs(sessions_df['z_'+col]) > 3]
    print('Outliers (z>|3|):', len(outliers))
    display(outliers.head(10))

    # IQR method
    q1 = sessions_df[col].quantile(0.25)
    q3 = sessions_df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    iqr_out = sessions_df[(sessions_df[col] < lower) | (sessions_df[col] > upper)]
    print('Outliers by IQR:', len(iqr_out))

In [ ]:
# Gráficos de distribución y popularidad
import matplotlib.pyplot as plt

# Sessions per user
if 'user_id' in sessions_df.columns:
    sessions_per_user = sessions_df.groupby('user_id').size().rename('sessions_count')
    plt.figure()
    plt.hist(sessions_per_user.values, bins=40)
    plt.title('Distribución de sesiones por usuario')
    plt.xlabel('Número de sesiones')
    plt.ylabel('Usuarios')
    plt.show()

# Top content by sessions
if 'content_id' in sessions_df.columns:
    content_counts = sessions_df['content_id'].value_counts().rename_axis('content_id').reset_index(name='session_views')
    if 'content_id' in content_df.columns and 'title' in content_df.columns:
        content_counts = content_counts.merge(content_df[['content_id','title']], on='content_id', how='left')
    top10 = content_counts.head(10)
    plt.figure()
    plt.barh(top10['title'].fillna(top10['content_id']), top10['session_views'])
    plt.title('Top 10 contenido por sesiones')
    plt.xlabel('Sesiones')
    plt.show()

# Completion rate distribution
if 'completion_rate' in sessions_df.columns:
    plt.figure()
    plt.hist(sessions_df['completion_rate'].dropna(), bins=30)
    plt.title('Distribución de completion_rate')
    plt.xlabel('Completion rate (0-1)')
    plt.show()

# Device usage
device_col_candidates = ['device','device_type','platform']
device_col = next((c for c in device_col_candidates if c in sessions_df.columns), None)
if device_col:
    device_counts = sessions_df[device_col].value_counts().head(10)
    device_counts.plot(kind='bar')
    plt.title('Top dispositivos / plataformas')
    plt.show()

# Geo preferences
geo_col_candidates = ['country','country_code','region','location']
geo_col = next((c for c in geo_col_candidates if c in sessions_df.columns), None)
if geo_col:
    geo_counts = sessions_df[geo_col].value_counts().head(10)
    geo_counts.plot(kind='bar')
    plt.title('Top localidades por sesiones')
    plt.show()

In [ ]:
# Data Quality Report
dq = pd.DataFrame({
    'n_missing': sessions_df.isna().sum(),
    'pct_missing': sessions_df.isna().mean()
}).sort_values('pct_missing', ascending=False)
display(dq.head(50))

print('Duplicated rows in sessions:', sessions_df.duplicated().sum())
if 'start_time' in sessions_df.columns:
    print('Date range:', sessions_df['start_time'].min(), sessions_df['start_time'].max())

In [ ]:
# Hypothesis test: Premium vs Basic (session_duration_min)
if ('user_id' in sessions_df.columns) and ('user_id' in users_df.columns) and ('subscription_type' in users_df.columns) and ('session_duration_min' in sessions_df.columns):
    merged = sessions_df.merge(users_df[['user_id','subscription_type']], on='user_id', how='left')
    premium = merged.loc[merged['subscription_type']=='Premium', 'session_duration_min'].dropna()
    basic = merged.loc[merged['subscription_type']=='Basic', 'session_duration_min'].dropna()
    print('N premium:', len(premium), 'N basic:', len(basic))

    levene_stat, levene_p = stats.levene(premium, basic, center='median')
    print('Levene p-value:', levene_p)

    equal_var = (levene_p > 0.05)
    t_stat, p_val = stats.ttest_ind(premium, basic, equal_var=equal_var, nan_policy='omit')
    print('t-stat:', t_stat, 'p-value:', p_val)

    def cohens_d(a,b):
        na, nb = len(a), len(b)
        sa, sb = a.std(ddof=1), b.std(ddof=1)
        s = np.sqrt(((na-1)*sa*sa + (nb-1)*sb*sb) / (na+nb-2))
        return (a.mean()-b.mean())/s
    try:
        print("Cohen's d:", cohens_d(premium, basic))
    except Exception as e:
        print('Cohen computation error:', e)
else:
    print('Faltan columnas para hacer la prueba.')

In [ ]:
# Correlation analysis: age, session_duration_min, completion_rate
if ('user_id' in sessions_df.columns) and ('age' in users_df.columns) and ('session_duration_min' in sessions_df.columns):
    merged = sessions_df.merge(users_df[['user_id','age']], on='user_id', how='left')
    corr_df = merged[['age','session_duration_min','completion_rate']].dropna()
    corr_mat = corr_df.corr(method='pearson')
    display(corr_mat)

    from itertools import combinations
    def pearson_pvalue(x,y):
        r, p = stats.pearsonr(x,y)
        return r, p

    for a,b in combinations(corr_df.columns,2):
        r,p = pearson_pvalue(corr_df[a], corr_df[b])
        print(f'Pearson r({a},{b})={r:.3f}, p={p:.3e}')
else:
    print('No hay columnas suficientes para correlación.')

In [ ]:
# Clustering: user-level aggregation and KMeans
agg = sessions_df.groupby('user_id').agg(
    sessions_count = ('session_id','count'),
    avg_duration = ('session_duration_min','mean'),
    avg_completion = ('completion_rate','mean')
).reset_index()

if 'user_id' in users_df.columns and 'age' in users_df.columns:
    agg = agg.merge(users_df[['user_id','age']], on='user_id', how='left')

features = ['sessions_count','avg_duration','avg_completion','age']
X = agg[features].fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Elbow plot
inertia = []
K = range(1,8)
for k in K:
    km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X_scaled)
    inertia.append(km.inertia_)
plt.figure()
plt.plot(list(K), inertia, '-o')
plt.xlabel('k'); plt.ylabel('Inertia'); plt.title('Elbow method'); plt.show()

# Silhouette
from sklearn.metrics import silhouette_score
for k in range(2,6):
    km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X_scaled)
    print('k=',k,'silhouette=', silhouette_score(X_scaled, km.labels_))

# Fit k=3 example
k = 3
km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X_scaled)
agg['cluster_kmeans'] = km.labels_
display(agg.groupby('cluster_kmeans').agg({
    'sessions_count':'mean','avg_duration':'mean','avg_completion':'mean','age':'mean','user_id':'count'
}).rename(columns={'user_id':'n_users'}))

# Dendrogram (muestra)
sample = X_scaled[:500]
Z = linkage(sample, method='ward')
plt.figure(figsize=(12,5))
dendrogram(Z, truncate_mode='level', p=5)
plt.title('Dendrogram (sample)')
plt.show()

In [ ]:
# Predictive model: simple RandomForest to predict retention
# Define retained as sessions_count >= 5 (example)
agg['retained'] = (agg['sessions_count'] >= 5).astype(int)
features = ['sessions_count','avg_duration','avg_completion','age']
X = agg[features].fillna(0)
y = agg['retained']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
try:
    print('ROC AUC:', roc_auc_score(y_test, y_prob))
except Exception as e:
    print('ROC AUC error:', e)

importances = pd.Series(clf.feature_importances_, index=features).sort_values(ascending=False)
display(importances)

In [ ]:
# Time series: sessions per week and seasonal decomposition
if 'start_time' in sessions_df.columns:
    ts = sessions_df.copy()
    ts['date'] = pd.to_datetime(ts['start_time']).dt.date
    daily = ts.groupby('date').size().rename('sessions').to_frame()
    daily.index = pd.to_datetime(daily.index)
    weekly = daily['sessions'].resample('W').sum()
    plt.figure()
    weekly.plot(title='Sessions per week')
    plt.ylabel('Sessions')
    plt.show()

    # seasonal decomposition
    try:
        from statsmodels.tsa.seasonal import seasonal_decompose
        res = seasonal_decompose(weekly, period=52, model='additive', two_sided=False)
        res.plot()
        plt.show()
    except Exception as e:
        print('Seasonal decomposition error (maybe series too short):', e)
else:
    print('No start_time column for timeseries.')

In [ ]:
# Guardar muestra limpia y wrap-up
out_path = os.path.join(DATA_PATH, 'sessions_clean_sample.csv')
sessions_df.head(2000).to_csv(out_path, index=False)
print('Saved sample to:', out_path)

# Save notebook summary (data quality and next steps)
print('\n--- Summary of artifacts ---')
print('- Descriptive stats, outliers, distribution plots')
print('- Hypothesis test results (Premium vs Basic)')
print('- Correlation matrix')
print('- Clustering assignment and profiles')
print('- Predictive model evaluation (classification)')
print('- Time series plots')
